# declaring a simulation setup for my MPC simulation

In [23]:
import numpy as np
import joblib

In [24]:
matrix_C = False
# Load matrices A, B, and C 
A = np.load('../data/A_C_' + str(matrix_C) + '.npy')
B = np.load('../data/B_C_' + str(matrix_C) + '.npy')
C = np.load('../data/C_C_' + str(matrix_C) + '.npy')

nz, nu = B.shape  # state and input dimensions
ny = C.shape[0]  # output dimensions

# disturbance 
F = np.eye(ny)
nd = F.shape[1]

scaler = joblib.load('../data/scaler.pkl')
scalerU = joblib.load('../data/scalerU.pkl')

In [25]:
# begining
y_start = np.array([0.5, 0.5])
y_start = scaler.transform(y_start.reshape(1, -1))

u_previous = np.array([0.2, 0.5])
u_previous = scalerU.transform(u_previous.reshape(1, -1))[0]

# observer
P0 = np.eye(nz+nd) 
Q = np.eye(nz+nd) * 0.1 
R = np.eye(ny) * 0.5

# controller
N = 20
Qy = np.eye(ny) * 5
Qu = np.eye(nu) * 0.1
u_min = scalerU.transform(np.array([[0.0, 0.0]]))[0]
u_max = scalerU.transform(np.array([[0.5, 1.0]]))[0]
y_min = scaler.transform(np.array([[0.0, 0.0]]))[0]
y_max = scaler.transform(np.array([[5.0, 5.0]]))[0]

# simulation
sim_time = 500

In [26]:
# reference trajectory

# Parameters
num_signals = ny
num_samples = sim_time
change_interval = 100


# Initialize reference array
reference = np.zeros((num_signals, num_samples))
ref = np.array([[0.5, 0.5],[1.5, 0.8],[1.0, 0.9],[2.0, 2.0],[1.0, 0.9]])

# Generate new value every `change_interval` steps
for i in range(0, num_samples, change_interval):
    new_value = ref[i // change_interval].reshape(-1,1)  
    reference[:, i:i+change_interval] = new_value

# Optional: Clip in case final interval exceeds array length
reference = reference[:, :num_samples]
reference = scaler.transform(reference.T).T

In [27]:
reference[:,100]

array([ 0.29135356, -0.22285209])

In [28]:
sim_setup = {
    'y_start': y_start,
    'u_previous': u_previous,
    'P0': P0,
    'Q': Q,
    'R': R,
    'N': N,
    'Qy': Qy,
    'Qu': Qu,
    'u_min': u_min,
    'u_max': u_max,
    'y_min': y_min,
    'y_max': y_max,
    'sim_time': sim_time,
    'reference': reference,
}

In [29]:
joblib.dump(sim_setup, "sim_setup.pkl")

['sim_setup.pkl']